In [7]:
import os
os.startfile(r"C:\\Users\\rayan\\OneDrive\\Bureau\\HackProject\\CellsAI\\Game\\deadcells.exe") 


In [3]:
import pyautogui
import time
import keyboard
# Wait for a few seconds to ensure the game has loaded
time.sleep(5)

def goRight():
    pyautogui.keyDown('right')
    time.sleep(2)
    pyautogui.keyUp('right')

def goLeft():
    pyautogui.keyDown('left')
    time.sleep(2)
    pyautogui.keyUp('left')

def jump():
    pyautogui.keyDown('space')
    time.sleep(1)
    pyautogui.keyUp('space')

def crouch():
    pyautogui.keyDown('down')
    time.sleep(1)
    pyautogui.keyUp('down')


def doubleJump():
    jump()
    jump()
    

def quickRight():
    goRight()
    time.sleep(0.1)
    goRight()

def quickLeft():
    goLeft()
    time.sleep(0.1)
    goLeft()

def dashRight():
    quickRight()
    quickRight()
    
def dashLeft():
    quickLeft()
    quickLeft()

    
def climb():
    pyautogui.keyDown('up')
    time.sleep(1)
    pyautogui.keyUp('up')

def roll():
    pyautogui.keyDown('ctrl')
    time.sleep(1)
    pyautogui.keyUp('ctrl')

def weapon1():
    pyautogui.keyDown('1')
    time.sleep(1)
                #left-click
    pyautogui.keyUp('1')

def weapon2():
    pyautogui.keyDown('2')
                            #right-click
    time.sleep(1)
    pyautogui.keyUp('2')

def health():
    pyautogui.keyDown('h')
    time.sleep(1)
    pyautogui.keyUp('h')

def collect():
    pyautogui.keyDown('r')
    time.sleep(1)
    pyautogui.keyUp('r')


    






In [4]:
def jump():
    pyautogui.keyDown('space')
    time.sleep(1)
    pyautogui.keyUp('space')

def Quickjump():
    pyautogui.keyDown('space')
    time.sleep(0.05)
    pyautogui.keyUp('space')

def doubleJump():
    Quickjump()
    Quickjump()
    
def hardStomp():
    pyautogui.keyDown('down')
    time.sleep(0.05)
    doubleJump()
    pyautogui.keyUp('down')

def jumpDown():
    pyautogui.keyDown('down')
    time.sleep(0.05)
    jump()
    pyautogui.keyUp('down')
    
while not keyboard.is_pressed('backspace'):
    time.sleep(3)
    dashRight()
    time.sleep(3)
    jumpDown()
    time.sleep(3)
    doubleJump()
    time.sleep(3)
    hardStomp()
    roll()


KeyboardInterrupt: 

In [5]:
while not keyboard.is_pressed('backspace'):
    doubleJump()
    roll()
    doubleJump()
    doubleJump()
    doubleJump()
    doubleJump()
    doubleJump() 
    crouch()
    dashRight()
    goLeft()
    dashRight()
    goLeft()
    dashRight()
    goLeft()    
    doubleJump()
    roll()
    doubleJump()
    doubleJump()
    doubleJump()
    doubleJump()
    doubleJump()
      doubleJump()
    
      
    climb()
    roll()
    doubleJump()
     
    weapon1()
    weapon2()
    #doubleJump()
    
    health()
    collect()
    
    goRight()
    #doubleJump()
    
    goRight()
    goRight()
    jump()
    jump()
    goLeft()
    jump()
    goRight()
    pass
    

IndentationError: unexpected indent (3076636357.py, line 23)